# Autoencoder

### Libraries

##### Standardbibliotheken
- os: ermöglicht Erstellung von Ordnern und Navigation durch die Ordnerstruktur des Projektes
- sys: sys.path.append erlaubt das Importieren von Klassen, welche außerhalb definiert sind (im src-Ordner)
- torch & torch.nn: Standard Pytorch Klassen
- torchvision datasets & transforms: ermöglicht Import von den Datensets (MNIST & CIFAR10) sowie die Transformation dieser
- matplotlib.pyplot: ermöglicht das Plotten der Bilder / Ergebnisse


##### Custom Imports
- classes.autoencoder.XY: importiert die jeweiligen AutoEncoder Klassen die im Ordner src/classes/autoencoder definiert sind.
- Klassen / Funktionen aus dem Ordner src/utils/ sind selbsterstelle Klassen / Funktionen, welche Code kapseln und so die Lesbarkeit verbessern. Die Dokumentation zu den Klassen / Funktionen findet sich in den entsprechenden Dateien


In [ ]:
import os
import sys
sys.path.append('../src')
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# custom imports
from classes.autoencoder.autoencoder import AutoEncoder
from classes.autoencoder.autoencoder4x100 import AutoEncoder4x100
from classes.autoencoder.autoencoderCNN import AutoEncoderCNN

from utils.foldergen import generate_folder
from utils.dataset import load_datasets
from utils.tracker import Tracker
from utils.drawImgs import view_reconstructed

### Training Configuration

Im Folgenden wird das Gerät konfiguriert auf dem die Berechnungen ausgeführt werden.
Zusätzlich lassen sich folgende Parameter konfigurieren:
- ``datasettype``: Auswahl zwischen "MNIST" oder "CIFAR10"
- ``continue_training``: Falls False wird das Training von vorne gestartet, falls True wird ein vorher gespeichertes Model aus "autoencoder.pth" geladen und weiter trainiert.
- ``evaluate``: Option, ob am Ende eine Evaluation statt finden soll

- ``num_epochs``: Die Anzahl der Epochen die trainiert werden
- ``batch_size``: \*
- ``learning_rate``: Die Lernrate

- ``graph_every_epoch``: konfiguriert welche N.te Epoche der Lossgraph erzeugt wird
- ``comparision_every_epoch``: konfiguriert wie oft ein Vergleich zwischen Input und Output des Autoencoder angezeigt wird

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

datasettype = "MNIST"
continue_training = False # Train a new network or continue training a previously trained network
evaluate = False

# Hyperparameter
num_epochs = 4
batch_size = 8
learning_rate = 1e-4

# output settings
graph_every_epoch = 1
comparision_every_epoch = 1

### Get Data

Im Folgenden wird die ``transform``-Funktion definiert, mit der die Daten transformiert werden.

Danach werden die Ordner erstellt, wo nachher der Output landet, falls diese noch nicht existieren.

Zum Schluss werden zwei Datasets und zwei Dataloader erzeugt, einmal mit dem Trainingsdatensatz, einmal mit dem Testdatenssatz.

In [ ]:
if datasettype == "MNIST":
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(
                [0.5], [0.5]
            ), 
        ]
    )
elif datasettype == "CIFAR10":
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

# create folder structure
folders = ["output", "save", "eval"]
generate_folder(folders)

dataset_train, loader_train = load_datasets(datasettype,transform, batch_size)
dataset_test, loader_test = load_datasets(datasettype,transform, batch_size, train=False, download=False)

### Network

Der Autoencoder wird als ``network`` implementiert.

Als ``lossFunction`` wird der Mean Squared Error verwendet.

Als ``optimizer`` wird der Adam-Optimizer verwendet, mit der obengesetzen Lernrate und einem ``weight_decay`` von 1e-8 

In [ ]:
network = AutoEncoderCNN(datasettype=datasettype).to(device)

lossFunction = nn.MSELoss()

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay= 1e-8)

### Model loading

Es wird die Trackerklasse implementiert, diese trackt den kompletten Lernprozess des Netwerkes und erzeugt später den Lossplot.
Ist ``continue_training`` auf True gesetzt, so wird das bereits trainierte Autoencoder-Model geladen und der Tracker läd die Daten, des vorherigen Trainings.

Außerdem wird geladen wie viele Epochen bereits absolviert wurden und die ``start_epoch`` wird entsprechend gesetzt. So wird später das Überschreiben von Bildern / Ergebnissen vermieden.

In [ ]:
tracker = Tracker()
if continue_training:
    network = torch.load('autoencoder.pth')
    tracker.load("data.json")
    start_epoch = tracker.epochs_completed + 1
else:
    start_epoch = 1

### Training

Die Trainingsschleife startet bei ``start_epoch`` und endet bei ``end_epoch`` (=``start_epoch+num_epochs``).

In [ ]:
end_epoch = start_epoch + num_epochs
for epoch in range(start_epoch, end_epoch):
    total_loss = 0

    tracker.epochs_completed += 1
    average_loss = 0.0
    eval_loss = 0
    
    for image, label in loader_train:

        image = image.to(device)
    
        # pass image through autoencoder
        reconstructed = network(image)

        # evaluate loss by comparing reconstructed image with actual image
        loss = lossFunction(reconstructed, image)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_size

    average_loss += total_loss / 60000
    print(epoch, "/", end_epoch-1, average_loss)
    tracker.y_loss["train"].append(average_loss)

    tracker.x_epoch.append(epoch)
    tracker.learning_rate.append(learning_rate)
    
    # Evaluation
    network.cpu()
    network.eval()
    image = image.to('cpu')

    reconstructed = network(image)

    eval_loss = lossFunction(reconstructed, image)
    # print(eval_loss.item())
    tracker.y_loss["val"].append(eval_loss.item())

    if epoch % comparision_every_epoch == 0:
        view_reconstructed(image, reconstructed)
        print(label[0].shape)

    image = image.to(device)
    network.to(device)
    network.train()
    torch.save(network, f'save/ep{epoch}.pth') # Save model at end of epoch

    # plot loss graph every given epoch
    if epoch % graph_every_epoch == 0:
        tracker.plotLossGraph()

# plot loss graph at the end of the last epoch, if it has not been printed yet
if not epoch % graph_every_epoch == 0:
    tracker.plotLossGraph()

### Save model

In [ ]:
torch.save(network, 'autoencoder.pth')
tracker.save("data.json")

### Evaluation

In [ ]:
# only works with DFF
if evaluate and datasettype == "MNIST":
    network = torch.load('autoencoder.pth')

    dataset_test = datasets.MNIST(
        root='../data', train=False, transform=transform, download=False
    )

    loader_test = torch.utils.data.DataLoader(
        dataset=dataset_test, batch_size=batch_size, shuffle=False
    )

    with torch.no_grad():
        for batch_idx, (image, _) in enumerate(loader_test):
            # take image from loader an flatten it
            image = image.reshape(-1, 28 * 28).to(device)

            # pass (flattened) image through autoencoder
            reconstructed = network(image)

            # evaluate loss by comparing reconstructed image with actual image
            loss = lossFunction(reconstructed, image)

            # Show input and reconstructed images side by side
            if batch_idx % 1000:
                fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
                axes[0].imshow(image[0].reshape(28, 28).to("cpu"), cmap="gray")
                axes[0].axis("off")
                axes[0].set_title("Input Image")
                axes[1].imshow(
                    reconstructed[0].detach().to("cpu").numpy().reshape(28, 28), cmap="gray"
                )
                axes[1].axis("off")
                axes[1].set_title("Reconstructed Image")
                plt.tight_layout()
                plt.savefig(os.path.join("eval/", f"test_{batch_idx}.png"))  # Save the figure
                plt.show()
                plt.close()
